In [1]:
import numpy as np
import pandas as pd
from tabulate import tabulate as tabulate
import pr_z_v11 as pr_z
import importlib
importlib.reload(pr_z)
R, f2r = 10.731577089016, 459.67

In [3]:
# Lee, Gonzalez & Eakin (1966) Correlation using equations 2.14-2.17 from 'Petroleum Reservoir Fluid Property Correlations' by W. McCain et al.
def gas_vis(degf, psia, z, mw):
    degR = degf + f2r
    rho = mw * psia / (degR * z * R ) # lb/cuft
    b = 3.448 + (986.4 / degR) + (0.01009 * mw)                                   # 2.16
    c = 2.447 - (0.2224 * b)                                                      # 2.17
    a = ((9.379 + (0.01607 * mw)) * degR ** 1.5 / (209.2 + (19.26 * mw) + degR))  # 2.15
    return a * 0.0001 * np.exp(b * (rho / 62.428) ** c)                           # 2.14

In [4]:
sgs = np.arange(0.5, 1.6, 0.1)
mws = sgs * 28.97
psias = np.arange(15, 15015, 200)
degfs = np.arange(60, 300, 25)
mwAir = 28.97

In [5]:
def tc_pc(sg_hc):
    v1, v2, v3 = 8.02982E-05, 0.056591187, 0.767839091
    Offset, Pl, Vl = 11.1719738, 183.6060878, 2657.114838
 
    mw_gas = mwAir * sg_hc
    tpc_hc = (mw_gas + Offset)*Vl / (mw_gas + Offset + Pl)
    Vc = v1 * mw_gas ** 2 + v2 * mw_gas + v3
    ppc_hc = 0.3074*10.7316*tpc_hc/Vc
    
    return (tpc_hc, ppc_hc)

In [7]:
out_string = ''
for i, mw in enumerate(mws):
    tpc_hc, ppc_hc = tc_pc(sgs[i])
    sg = round(sgs[i],1)
    name = str(sg).replace('.', 'p')
    out_string += 'CHARACTERIZATION "CHR_'+name+'" BASE "RealGas"\n'
    out_string += 'COMPONENT    MW      TC,R     PC,PSIA    AF    VTRAN    A    B    AMOD    BMOD      VCVIS(ft3/lbmol)\n'
    out_string += '-----------  ------  -------  ---------  ----  -------  ---  ---  ------  ------    ----------------\n'
    out_string += 'Gas          '+str(round(mw,4))+'  '+str(round(tpc_hc,4))+'  '+str(round(ppc_hc,4))+'\nEND\n\n'

    guess_v = 0.056147017 * mw + 0.500362727

    print('VARIABLE vc_'+name+' '+str(round(guess_v, 3))+'  0.1  5 ; '+str(mw))
    
    out_string += 'REPLACE VCVIS of Gas by vc_'+name+'  ft3/lbmol\n\n'
    out_string += 'QIX Gas_'+name+'\n'
    out_string += '---  ------\n'
    out_string += 'Gas  1.0\n'
    out_string += '---  ------\n'
    out_string += 'END\n\n'

    out_string += 'QIX FEED 1 MOLE Gas_'+name+'\n'
    out_string += 'CCE ID "Viscosity '+str(sg)+' SG"\n'

    temps, press, zss, viscs = [[] for x in range(4)]    
    for degf in degfs:
        for psia in psias:
            z = pr_z.peng_robinson_z(degf, psia, sg, co2=0, h2s=0, n2=0, h2 = 0)
            vis = gas_vis(degf, psia, z, mw)
            temps.append(degf)
            press.append(psia)
            zss.append(round(z,6))
            viscs.append(round(vis,6))
    temps.append('Wt')
    press.append('')
    zss.append('0')
    viscs.append('1.0')
    
    exp_df = pd.DataFrame()
    exp_df['TEMP (F)'] = temps
    exp_df['PRES (psia)'] = press
    exp_df['GZ'] = zss
    exp_df['GVIS (cp)'] = viscs
    
    out_string += tabulate(exp_df, headers='keys', showindex='never', numalign='left')
    out_string += '\nEND\n\n'

with open('gas_viscosity_v11.inc', 'w') as file:
            file.write(out_string)


VARIABLE vc_0p5 1.314  0.1  5 ; 14.485
VARIABLE vc_0p6 1.476  0.1  5 ; 17.381999999999998
VARIABLE vc_0p7 1.639  0.1  5 ; 20.278999999999996
VARIABLE vc_0p8 1.802  0.1  5 ; 23.176
VARIABLE vc_0p9 1.964  0.1  5 ; 26.072999999999997
VARIABLE vc_1p0 2.127  0.1  5 ; 28.969999999999995
VARIABLE vc_1p1 2.29  0.1  5 ; 31.866999999999994
VARIABLE vc_1p2 2.452  0.1  5 ; 34.76399999999999
VARIABLE vc_1p3 2.615  0.1  5 ; 37.660999999999994
VARIABLE vc_1p4 2.778  0.1  5 ; 40.55799999999999
VARIABLE vc_1p5 2.94  0.1  5 ; 43.45499999999999


## Result
## Vc_Vis = 0.223276698 * MW ** 0.694183122

======  NOTE: Standard LBC correlation now used for viscosities  ======             
P0 = 0.1023, P1 = 0.023364, P2 = 0.058533, P3 = -0.040758, P4 = 0.0093324, F0 = 0.1 

